# Amazon SageMaker Clarify Model Bias Monitor for Batch Transform

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

---

## Runtime

This notebook takes approximately 60 minutes to run.

## Contents

* [Introduction](#Introduction)
* [General Setup](#General-Setup)
    * [Imports](#Imports)
    * [Handful of configuration](#Handful-of-configuration)
    * [Data files](#Data-files)
    * [SageMaker model](#SageMaker-model)
* [Batch Transform Job](#Batch-Transform-Job)
    * [Captured data](#Captured-data)
    * [Transform output](#Transform-output)
* [Ground Truth Data](#Ground-Truth-Data)
* [Model Bias Monitor](#Model-Bias-Monitor)
    * [Baselining job](#Baselining-job)
        * [Configurations](#Configurations)
        * [Kick off baselining job](#Kick-off-baselining-job)
    * [Monitoring Schedule](#Monitoring-Schedule)
        * [Wait for the first execution](#Wait-for-the-first-execution)
        * [Wait for the execution to finish](#Wait-for-the-execution-to-finish)
        * [Merged data](#Merged-data)
        * [Inspect execution results](#Inspect-execution-results)
* [Cleanup](#Cleanup)

## Introduction

[Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) continuously monitors the quality of Amazon SageMaker machine learning models in production. It enables developers to set alerts for when there are deviations in the model quality. Early and pro-active detection of these deviations enables corrective actions, such as retraining models, auditing upstream systems, or fixing data quality issues without having to monitor models manually or build additional tooling. 

[Amazon SageMaker Clarify Model Bias Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift.html) is a model monitor that helps data scientists and ML engineers monitor predictions for bias on a regular basis. Bias can be introduced or exacerbated in deployed ML models when the training data differs from the data that the model sees during deployment (that is, the live data). These kinds of changes in the live data distribution might be temporary (for example, due to some short-lived, real-world events) or permanent. In either case, it might be important to detect these changes. For example, the outputs of a model for predicting home prices can become biased if the mortgage rates used to train the model differ from current, real-world mortgage rates. With bias drift detection capabilities in model monitor, when SageMaker detects bias beyond a certain threshold, it automatically generates metrics that you can view in SageMaker Studio and through Amazon CloudWatch alerts. 

This notebook demonstrates the process for setting up a model monitor for continuous monitoring of bias drift of the data and model used by a regularly running [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) job. The model input and output are in CSV format.

In general, you can use the model bias monitor for batch transform in this way,

1. Schedule a model bias monitor to monitor a data capture S3 location and a ground truth S3 location
1. Regularly run transform jobs with data capture enabled, the jobs save captured data to the data capture S3 URI
1. Regularly label the captured data, and then upload the ground truth labels to the ground truth S3 URI

The monitor executes processing jobs regularly to merge the captured data and ground truth data, do bias analysis for the merged data, and then generate analysis reports and publish metrics to CloudWatch.

## General Setup

The notebook uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk). The following cell upgrades the SDK and its dependencies. Then you may need to restart the kernel and rerun the notebook to pick up the up-to-date APIs, if the notebook is executed in the SageMaker Studio.

In [ ]:
!pip install -U sagemaker
!pip install -U boto3
!pip install -U botocore

### Imports

The following cell imports the APIs to be used by the notebook.

In [2]:
import sagemaker
import pandas as pd
import datetime
import io
import json
import os
import random
import time
import pprint

### Handful of configuration

To begin, ensure that these prerequisites have been completed.

* Specify an AWS Region to host the model.
* Specify an IAM role to execute jobs.
* Define the S3 URIs that stores the model file, input data and output data. For demonstration purposes, this notebook uses the same bucket for them. In reality, they could be separated with different security policies.

In [3]:
sagemaker_session = sagemaker.Session()

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

role = sagemaker.get_execution_role()
print(f"RoleArn: {role}")

# A different bucket can be used, but make sure the role for this notebook has
# the s3:PutObject permissions. This is the bucket into which the data is captured
bucket = sagemaker_session.default_bucket()
print(f"Demo Bucket: {bucket}")
prefix = sagemaker.utils.unique_name_from_base("sagemaker/DEMO-ClarifyModelMonitor")
print(f"Demo Prefix: {prefix}")
s3_key = f"s3://{bucket}/{prefix}"
print(f"Demo S3 key: {s3_key}")

data_capture_s3_uri = f"{s3_key}/data-capture"
ground_truth_s3_uri = f"{s3_key}/ground-truth"
transform_output_s3_uri = f"{s3_key}/transform-output"
baselining_output_s3_uri = f"{s3_key}/baselining-output"
monitor_output_s3_uri = f"{s3_key}/monitor-output"

print(f"The transform job will save the results to: {transform_output_s3_uri}")
print(f"The transform job will save the captured data to: {data_capture_s3_uri}")
print(f"You should upload the ground truth data to: {ground_truth_s3_uri}")
print(f"The baselining job will save the analysis results to: {baselining_output_s3_uri}")
print(f"The monitor will save the analysis results to: {monitor_output_s3_uri}")

AWS region: us-west-2
RoleArn: arn:aws:iam::000000000000:role/service-role/AmazonSageMaker-ExecutionRole-20200714T163791
Demo Bucket: sagemaker-us-west-2-000000000000
Demo Prefix: sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2
Demo S3 key: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2
The transform job will save the results to: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/transform-output
The transform job will save the captured data to: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/data-capture
You should upload the ground truth data to: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/ground-truth
The baselining job will save the analysis results to: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/baselining-output
The monitor will save the analysis results to: s3://sagema

### Data files

This example includes two dataset files, both in CSV format.

* The train dataset has header row, and it has a target column followed by the feature columns.
* The test dataset is not headers, and it only has feature columns.

In [4]:
train_dataset_path = "test_data/validation-dataset-with-header.csv"
test_dataset_path = "test_data/test-dataset-input-cols.csv"
dataset_type = "text/csv"

In [5]:
df = pd.read_csv(train_dataset_path)
df.head(5)

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,...,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,0,47,28,141.3,94,168.0,108,113.5,84,7.8,...,0,0,0,1,0,0,1,0,0,1
1,0,30,0,247.4,107,175.9,76,287.4,90,11.3,...,0,0,0,0,1,0,1,0,1,0
2,0,106,32,165.9,126,216.5,93,173.1,86,14.1,...,0,0,0,1,0,0,1,0,0,1
3,0,131,0,240.9,108,167.4,91,322.2,109,14.7,...,0,0,0,1,0,0,1,0,1,0
4,0,83,37,78.5,109,210.5,101,179.7,102,11.8,...,0,1,0,0,1,0,1,0,0,1


In [6]:
# Read headers
all_headers = list(df.columns)
label_header = all_headers[0]

To verify that the execution role for this notebook has the necessary permissions to proceed, put a simple test object into the S3 bucket specified above. If this command fails, update the role to have `s3:PutObject` permission on the bucket and try again.

In [7]:
sagemaker.s3.S3Uploader.upload_string_as_file_body(
    body="hello",
    desired_s3_uri=f"{s3_key}/upload-test-file.txt",
    sagemaker_session=sagemaker_session,
)
print("Success! We are all set to proceed with uploading to S3.")

Success! We are all set to proceed with uploading to S3.


Then upload the data files to S3 so that they can be used by SageMaker jobs.

In [8]:
train_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Train data is uploaded to: {train_data_s3_uri}")
test_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Test data is uploaded to: {test_data_s3_uri}")

Train data is uploaded to: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/validation-dataset-with-header.csv
Test data is uploaded to: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/test-dataset-input-cols.csv


### SageMaker model

This example includes a pre-built [SageMaker XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) model file trained by [XGBoost Churn Prediction Notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb). The following cell uploads the file to S3 and then creates a SageMaker model using it. The model support CSV data format, the input are customer attributes, and the output is the probability of customer churn (a float number between zero and one).

In [9]:
model_file = "model/xgb-churn-prediction-model.tar.gz"
model_url = sagemaker.s3.S3Uploader.upload(
    local_path=model_file,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Model file has been uploaded to {model_url}")

model_name = sagemaker.utils.unique_name_from_base("DEMO-xgb-churn-pred-model-monitor")
print(f"SageMaker model name: {model_name}")

image_uri = sagemaker.image_uris.retrieve("xgboost", region, "0.90-1")
print(f"SageMaker XGBoost image: {image_uri}")

model = sagemaker.model.Model(image_uri=image_uri, model_data=model_url, role=role)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)
print("SageMaker model created")

Model file has been uploaded to s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/xgb-churn-prediction-model.tar.gz
SageMaker model name: DEMO-xgb-churn-pred-model-monitor-1674106772-58f3
SageMaker XGBoost image: 246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3
SageMaker model created


## Batch Transform Job

For continuous monitoring, batch transform jobs should be executed regularly with the latest data. But for demonstration purpose, the following cell only executes the job once before the monitor is scheduled, so that the first monitoring execution has captured data to process. 

See [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) for the API reference. Highlights,

* `destination_s3_uri` is used to specify the data capture S3 URI which is a key connection between the job and the monitor.
* `join_source` must be set to "Input" for the transform output to include predictions (model output) as well as features (model input), because model bias monitor requires both.
* `generate_inference_id` must be set to True for the transform output to include a unique ID for each record. Model bias monitor requires both predicted labels and ground truth labels, so it needs the ID to join the captured data and the ground truth data.

**NOTE**: The following cell takes about 5 minutes to run.

In [10]:
transfomer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    accept=dataset_type,  # The transform output data format
    assemble_with="Line",  # CSV records are terminated by new lines
    output_path=transform_output_s3_uri,
)

transfomer.transform(
    data=test_data_s3_uri,
    content_type=dataset_type,  # The transform input format
    split_type="Line",  # CSV records are terminated by new lines
    join_source="Input",  # Include model input (features) in transform output
    batch_data_capture_config=sagemaker.inputs.BatchDataCaptureConfig(
        destination_s3_uri=data_capture_s3_uri,
        generate_inference_id=True,  # Inference ID is mandatory to join the captured data and the ground truth data
    ),
    wait=True,  # In real world you don't have to wait, but for demo purpose we wait for the output
    logs=False,  # You can change it to True to view job logs inline
)

.................................................!


### Captured data

Once the transform job completed, an "output" folders is created under `data_capture_s3_uri`, to includes the captured data files of transform output. Note that, batch transform data capture is unlike endpoint data capture, it does not capture the data for real as it will create tremendous amount of duplications. Instead, it generates [manifest](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html#sagemaker-Type-S3DataSource-S3Uri) files which refer to the transform output S3 location.

Now list the captured data files stored in Amazon S3. There should be different files from different time periods organized based on the hour in which the batch transformation occurred. The format of the Amazon S3 path is:

`s3://{data_capture_s3_uri}/output/yyyy/mm/dd/hh/filename.jsonl`

In [11]:
data_capture_output = f"{data_capture_s3_uri}/output"
captured_data_files = sorted(
    sagemaker.s3.S3Downloader.list(
        s3_uri=data_capture_output,
        sagemaker_session=sagemaker_session,
    )
)
print("Found capture data files:")
print("\n ".join(captured_data_files[-5:]))

Found capture data files:
s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/data-capture/output/2023/01/19/05/626c6b2e-23cd-4232-94c5-299e69009738.json


In [12]:
data_capture_output_dict = json.loads(
    sagemaker.s3.S3Downloader.read_file(
        s3_uri=captured_data_files[-1],
        sagemaker_session=sagemaker_session,
    )
)
print(json.dumps(data_capture_output_dict, indent=4))

[
    {
        "prefix": "s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/transform-output/"
    },
    "test-dataset-input-cols.csv.out"
]


### Transform output

The captured data file refers to the transform output `.out` file. The cell below shows the first few records of the file.

* The first columns are the feature columns (the model input), because the `join_source` parameter is set to "Input".
* Then there is the prediction column (the model output).
* The second last element is the inference ID, and the last is the inference time (the start time of the transform job). They are available because the `generate_inference_id` parameter is set to True.

In [13]:
transform_output = os.path.join(data_capture_output_dict[0]["prefix"], data_capture_output_dict[1])
transform_output_content = sagemaker.s3.S3Downloader.read_file(
    s3_uri=transform_output,
    sagemaker_session=sagemaker_session,
)
transform_output_df = pd.read_csv(io.StringIO(transform_output_content), header=None)
transform_output_df.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,...,0.52,0.53,1.2,1.3,0.54,1.4,0.55,0.015842,bd6c3be0-5902-4008-afc2-538b3be3bf2c,2023-01-19T05:40:55Z
1,132,25.0,113.2,96,269.9,107,229.1,87,7.1,7,...,0.00,1.00,0.0,1.0,0.00,0.0,1.00,0.010764,58cd9844-a37e-4d80-9372-319ca6d876f6,2023-01-19T05:40:55Z
2,112,17.0,183.2,95,252.8,125,156.7,95,9.7,3,...,0.00,1.00,0.0,1.0,0.00,0.0,1.00,0.008151,dba5c2f8-3203-4dd6-aa6e-69a4dd8279b7,2023-01-19T05:40:55Z
3,91,24.0,93.5,112,183.4,128,240.7,133,9.9,3,...,0.00,0.00,1.0,0.0,1.00,0.0,1.00,0.171978,09b7b0e0-fbd6-4168-8c43-9a384f363f8b,2023-01-19T05:40:55Z
4,22,0.0,110.3,107,166.5,93,202.3,96,9.5,5,...,1.00,0.00,0.0,1.0,0.00,1.0,0.00,0.007273,8d240e95-415b-428a-b669-368aa4945f10,2023-01-19T05:40:55Z


## Ground Truth Data

Besides captured data, bias drift monitoring execution also requires ground truth data. In real use cases, you should regularly label the captured data, then upload the ground truth data (labels) to designated S3 location. For demonstration purpose, this example notebook generates fake ground truth data following [this schema](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-merge.html), and then uploads it to `ground_truth_s3_uri` which is another key input to the monitor. The bias drift monitoring execution will first merge the captured data and the ground truth data, and then do bias analysis for the merged data.

In [14]:
def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    # format required by the merge job and bias monitoring job
    return {
        "groundTruthData": {
            "data": "1"
            if random.random() < 0.7
            else "0",  # randomly generate positive labels 70% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(
                inference_id
            ),  # the id is used to join the captured data and the ground truth data
        },
        "eventVersion": "0",
    }


def upload_ground_truth(upload_time, upload_path, inference_ids):
    records = [ground_truth_with_id(inference_id) for inference_id in inference_ids]
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    sagemaker.s3.S3Uploader.upload_string_as_file_body(
        body=data_to_upload,
        desired_s3_uri=target_s3_uri,
        sagemaker_session=sagemaker_session,
    )

In [15]:
now = datetime.datetime.utcnow()
inference_ids = list(transform_output_df.iloc[:, -2])  # get inference ID from the captured data
# Generate data for the last hour, in case the first monitoring execution is in this hour
upload_ground_truth(
    upload_time=now - datetime.timedelta(hours=1),
    upload_path=ground_truth_s3_uri,
    inference_ids=inference_ids,
)
# Generate data for this hour, in case the first monitoring execution will be in the next hour
upload_ground_truth(
    upload_time=now,
    upload_path=ground_truth_s3_uri,
    inference_ids=inference_ids,
)

Uploading 334 records to s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/ground-truth/2023/01/19/04/4341.jsonl
Uploading 334 records to s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/ground-truth/2023/01/19/05/4341.jsonl


## Model Bias Monitor

Similar to the other monitoring types, the standard procedure of creating a [bias drift monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift.html) is first run a baselining job, and then schedule the monitor.

A bias drift monitoring execution starts a merge job that joins the captured data and ground truth data together using the inference ID. Then a SageMaker Clarify bias analysis job is started to compute all the [pre-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html) and [post-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html). on the merged data. The max execution time is divided equally between two jobs, the notebook is scheduling an hourly model bias monitor, so the `max_runtime_in_seconds` parameter should not exceed 1800 seconds.

In [16]:
model_bias_monitor = sagemaker.model_monitor.ModelBiasMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

### Baselining job

A baselining job runs predictions on training dataset and suggests constraints. The `suggest_baseline()` method of `ModelBiasMonitor` starts a SageMaker Clarify processing job to generate the constraints.

The step is not mandatory, but providing constraints file to the monitor can enable violations file generation.

#### Configurations

Information about the input data need to be provided to the processor.

`DataConfig` stores information about the dataset to be analyzed. For example, the dataset file, its format (like CSV), headers and label.

In [17]:
data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_s3_uri,
    s3_output_path=baselining_output_s3_uri,
    label=label_header,
    headers=all_headers,
    dataset_type=dataset_type,
)

`ModelConfig` is configuration related to model to be used for inferencing. In order to compute post-training bias metrics, the computation needs to get inferences for the SageMaker model. To accomplish this, the processing job will use the model to create an ephemeral endpoint (also known as "shadow endpoint"). The processing job will delete the shadow endpoint after the computations are completed.

In [18]:
model_config = sagemaker.clarify.ModelConfig(
    model_name=model_name,  # The name of the SageMaker model
    instance_type="ml.m5.xlarge",  # The instance type of the shadow endpoint
    instance_count=1,  # The instance count of the shadow endpoint
    content_type=dataset_type,  # The data format of the model input
    accept_type=dataset_type,  # The data format of the model output
)

`ModelPredictedLabelConfig` specifies how to extract predicted label from the model output. The example model returns a single probability value between `0` and `1`. So,

* The `probability` parameter is set to zero, which is the index of the probability value in the CSV model output. 
* The `probability_threshold` parameter is used by post-training analysis to convert the probability/score to binary predicted label (`0` or `1`). The default value is 0.5. Here choose an arbitrary 0.8 cutoff, i.e. a probability value > 0.8 means customer will churn (predicted label `1`).

In [19]:
model_predicted_label_config = sagemaker.clarify.ModelPredictedLabelConfig(
    probability=0,  # The zero-based index of the probability (score) in model output
    probability_threshold=0.8,
)

`BiasConfig` is the configuration of the sensitive groups in the dataset. Typically, bias is measured by computing a metric and comparing it across groups. The group of interest is specified using the `facet` configuration. With the following facet, the baselining job will check if the model favors new customers (accounts created not far ago).

In [20]:
bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[1],
    facet_name="Account Length",
    facet_values_or_threshold=[100],
)

#### Kick off baselining job

Call the `suggest_baseline()` method to start the baselining job. The job computes all [pre-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html) and [post-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html).

In [21]:
model_bias_monitor.suggest_baseline(
    bias_config=bias_config,
    data_config=data_config,
    model_config=model_config,
    model_predicted_label_config=model_predicted_label_config,
)


Job Name:  baseline-suggestion-job-2023-01-19-05-43-42-284
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/validation-dataset-with-header.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/baselining-output/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/baselini

**NOTE**: The following cell waits until the baselining job is completed (in about 10 minutes). It then inspects the suggested constraints. This step can be skipped, because the monitor to be scheduled will automatically pick up baselining job name and wait for it before monitoring execution.

In [22]:
model_bias_monitor.latest_baselining_job.wait(logs=False)
print()
model_bias_constraints = model_bias_monitor.suggested_constraints()
print(f"Suggested constraints: {model_bias_constraints.file_s3_uri}")
print(
    sagemaker.s3.S3Downloader.read_file(
        s3_uri=model_bias_constraints.file_s3_uri,
        sagemaker_session=sagemaker_session,
    )
)

....................................................................................................!
Suggested constraints: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/baselining-output/analysis.json
{
    "version": "1.0",
    "post_training_bias_metrics": {
        "label": "Churn",
        "facets": {
            "Account Length": [
                {
                    "value_or_threshold": "(100, 225]",
                    "metrics": [
                        {
                            "name": "AD",
                            "description": "Accuracy Difference (AD)",
                            "value": 0.03416521605801226
                        },
                        {
                            "name": "CDDPL",
                            "description": "Conditional Demographic Disparity in Predicted Labels (CDDPL)",
                            "value": null,
                            "error": "Group variable is empty or

### Monitoring Schedule


With above constraints collected, now call `create_monitoring_schedule()` method to schedule an hourly model bias monitor.

If a baselining job has been submitted, then the monitor object will automatically pick up the analysis configuration from the baselining job. But if the baselining step is skipped, or if the capture dataset has different nature than the training dataset, then analysis configuration has to be provided.

`BiasAnalysisConfig` is a subset of the configuration of the baselining job, many options are not needed because,

* Model bias monitor will merge the captured data and the ground truth data, and then use the merged data as the input dataset.
* Capture data already includes predictions, so there is no need to create shadow endpoint.
* Attributes like probability threshold are provided as part of `BatchTransformInput`.

Highlights,

* `data_capture_s3_uri` is the location of data captured by the batch transform job
* `ground_truth_s3_uri` is the location of ground truth data
* `probability_attribute` stores the index of the probability value in model output. (Similar to the `probability` parameter of `ModelPredictedLabelConfig`.)
* `probability_threshold_attribute` is the same as the `probability_threshold` parameter of `ModelPredictedLabelConfig`.

In [23]:
schedule_expression = sagemaker.model_monitor.CronExpressionGenerator.hourly()

In [24]:
model_bias_analysis_config = None
if not model_bias_monitor.latest_baselining_job:
    model_bias_analysis_config = sagemaker.clarify.BiasAnalysisConfig(
        bias_config,
        headers=all_headers,
        label=label_header,
    )
model_bias_monitor.create_monitoring_schedule(
    analysis_config=model_bias_analysis_config,
    batch_transform_input=sagemaker.model_monitor.BatchTransformInput(
        data_captured_destination_s3_uri=data_capture_s3_uri,
        destination="/opt/ml/processing/transform",
        dataset_format=sagemaker.model_monitor.MonitoringDatasetFormat.csv(header=False),
        probability_attribute="0",
        probability_threshold_attribute=0.8,
        # look back 6 hour for transform job output.
        start_time_offset="-PT6H",
        end_time_offset="-PT0H",
    ),
    ground_truth_input=ground_truth_s3_uri,
    output_s3_uri=monitor_output_s3_uri,
    schedule_cron_expression=schedule_expression,
)
print(f"Model bias monitoring schedule: {model_bias_monitor.monitoring_schedule_name}")

Model bias monitoring schedule: monitoring-schedule-2023-01-19-05-52-05-332


#### Wait for the first execution

The schedule starts jobs at the previously specified intervals. Code below waits until time crosses the hour boundary (in UTC) to see executions kick off.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule executions. The execution might start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [25]:
def wait_for_execution_to_start(model_monitor):
    print(
        "An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(10)

    print()
    print("Done! Execution has started")

**NOTE**: The following cell waits until the first monitoring execution is started. As explained above, the wait could take more than 60 minutes.

In [26]:
wait_for_execution_to_start(model_bias_monitor)

An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
Waiting for the first execution to happen..........
Done! Execution has been created
Now waiting for execution to start.
Done! Execution has started


In real world, a monitoring schedule is supposed to be active all the time. But in this example, it can be stopped to avoid incurring extra charges. A stopped schedule will not trigger further executions, but the ongoing execution will continue. And if needed, the schedule can be restarted by `start_monitoring_schedule()`.

In [27]:
model_bias_monitor.stop_monitoring_schedule()


Stopping Monitoring Schedule with name: monitoring-schedule-2023-01-19-05-52-05-332


#### Wait for the execution to finish

In the previous cell, the first execution has started. This section waits for the execution to finish so that its analysis results are available. Here are the possible terminal states and what each of them mean:

* `Completed` - This means the monitoring execution completed, and no issues were found in the violations report.
* `CompletedWithViolations` - This means the execution completed, but constraint violations were detected.
* `Failed` - The monitoring execution failed, maybe due to client error (perhaps incorrect role permissions) or infrastructure issues. Further examination of `FailureReason` and `ExitMessage` is necessary to identify what exactly happened.
* `Stopped` - job exceeded max runtime or was manually stopped.

In [28]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            time.sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print(f"Done! Execution Status: {execution_summary['MonitoringExecutionStatus']}")
    else:
        print("Last execution not found")

**NOTE**: The following cell takes about 10 minutes.

In [29]:
wait_for_execution_to_finish(model_bias_monitor)

Waiting for execution to finish.........
Done! Execution Status: CompletedWithViolations


#### Merged data

Merged data is the intermediate results of bias drift monitoring execution. It is saved to JSON Lines files under the "merge" folder of `monitor_output_s3_uri`. Each line is a valid JSON object which combines the captured data and the ground truth data.

In [30]:
merged_data_s3_uri = f"{monitor_output_s3_uri}/merge"
merged_data_files = sorted(
    sagemaker.s3.S3Downloader.list(
        s3_uri=merged_data_s3_uri,
        sagemaker_session=sagemaker_session,
    )
)
print("Found merged data files:")
print("\n ".join(merged_data_files[-5:]))

Found merged data files:
s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/monitor-output/merge/monitoring-schedule-2023-01-19-05-52-05-332/2023/01/19/05/part-00000-93afd657-9941-49cb-9dc5-86ce85dee312.c000.jsonl


The following cell prints a single line of a merged data file.

* `eventId` is the inference ID from the captured data and the ground truth data
* `groundTruthData` is from the ground truth data
* `captureData` is from the captured data. In this case, the `data` of `batchTransformOutput` is from the transform output.

In [31]:
merged_record = sagemaker.s3.S3Downloader.read_file(
    s3_uri=merged_data_files[-1],
    sagemaker_session=sagemaker_session,
).splitlines()[0]
print(json.dumps(json.loads(merged_record), indent=4))

{
    "eventMetadata": {
        "eventId": "bd6c3be0-5902-4008-afc2-538b3be3bf2c"
    },
    "eventVersion": "0",
    "groundTruthData": {
        "data": "1",
        "encoding": "CSV"
    },
    "captureData": {
        "batchTransformOutput": {
            "data": "186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.4,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.5,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55,0.01584203727543354",
            "encoding": "CSV"
        }
    }
}


#### Inspect execution results

List the generated reports,

* analysis.json includes all the bias metrics.
* report.* files are static report files to visualize the bias metrics

In [32]:
schedule_desc = model_bias_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_bias_monitor_execution = model_bias_monitor.list_executions()[-1]
    last_model_bias_monitor_execution_report_uri = (
        last_model_bias_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_bias_monitor_execution_report_uri}")
    last_model_bias_monitor_execution_report_files = sorted(
        sagemaker.s3.S3Downloader.list(
            s3_uri=last_model_bias_monitor_execution_report_uri,
            sagemaker_session=sagemaker_session,
        )
    )
    print("Found Report Files:")
    print("\n ".join(last_model_bias_monitor_execution_report_files))
else:
    last_model_bias_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

Report URI: s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/monitor-output/monitoring-schedule-2023-01-19-05-52-05-332/2023/01/19/06
Found Report Files:
s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/monitor-output/monitoring-schedule-2023-01-19-05-52-05-332/2023/01/19/06/analysis.json
 s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/monitor-output/monitoring-schedule-2023-01-19-05-52-05-332/2023/01/19/06/constraint_violations.json
 s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/monitor-output/monitoring-schedule-2023-01-19-05-52-05-332/2023/01/19/06/report.html
 s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2/monitor-output/monitoring-schedule-2023-01-19-05-52-05-332/2023/01/19/06/report.ipynb
 s3://sagemaker-us-west-2-000000000000/sagemaker/DEMO-ClarifyModelMonitor-1674106771-acb2

If there are any violations compared to the baseline, they are listed here. See [Bias Drift Violations](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift-violations.html) for the schema of the file, and how violations are detected.

In [33]:
violations = model_bias_monitor.latest_monitoring_constraint_violations()
if violations is not None:
    pprint.PrettyPrinter(indent=4).pprint(violations.body_dict)

{   'version': '1.0',
    'violations': [   {   'constraint_check_type': 'bias_drift_check',
                          'description': 'Metric value -0.2777777777777778 '
                                         "doesn't meet the baseline constraint "
                                         'requirement 0.0',
                          'facet': 'Account Length',
                          'facet_value': '(100, 232]',
                          'metric_name': 'DAR'},
                      {   'constraint_check_type': 'bias_drift_check',
                          'description': 'Metric value 2.916666666666666 '
                                         "doesn't meet the baseline constraint "
                                         'requirement -0.4137931034482758',
                          'facet': 'Account Length',
                          'facet_value': '(100, 232]',
                          'metric_name': 'DCA'},
                      {   'constraint_check_type': 'bias_drift_check',
 

By default, the analysis results are also published to CloudWatch, see [CloudWatch Metrics for Bias Drift Analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-bias-drift-cw.html).

## Cleanup

If there is no plan to collect more data for bias drift monitoring, then the monitor should be stopped (and deleted) to avoid incurring additional charges. Note that deleting the monitor does not delete the data in S3.

In [34]:
model_bias_monitor.stop_monitoring_schedule()
wait_for_execution_to_finish(model_bias_monitor)
model_bias_monitor.delete_monitoring_schedule()
sagemaker_session.delete_model(model_name)


Stopping Monitoring Schedule with name: monitoring-schedule-2023-01-19-05-52-05-332
Waiting for execution to finish
Done! Execution Status: CompletedWithViolations

Deleting Monitoring Schedule with name: monitoring-schedule-2023-01-19-05-52-05-332


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)

![This badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker_model_monitor|fairness_and_explainability|SageMaker-Monitoring-Bias-Drift-for-Batch-Transform.ipynb)
